# WORDLE Breaker

### https://www.powerlanguage.co.uk/wordle/  

In [129]:
import re
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [130]:
def special_match(pattern, strg):
    result = re.match(pattern, strg)
    return result is not None


In [141]:
def break_wordle(attempts, word_lengh, dictionary):
   
    
    correct_pos_list = [] 
    incorrect_pos_list = [] 
    exist_in_list  = []
    not_exist_list = ["."]
    not_recognized_word_list = []    

    
    correct_pos_list = incorrect_pos_list  = (["a-z"] * word_lengh)

    correct_pos_rex   = "^"+ "".join( "[" + str(x) +"]" for x in correct_pos_list ) +"$"
    incorrect_pos_rex = "^"+ "".join( "[" + str(x) +"]" for x in incorrect_pos_list ) +"$"
    exist_in_rex      = "^.*" + "".join( "(?=.*[" + str(x) +"])" for x in exist_in_list )  +".*"
    not_exist_rex     = "^.*(?=.*[" + "".join(str(x) for x in not_exist_list )  +"]).*"
    

    for i in range(attempts):        
        print("\n###############################################################################################")
        print(f'''Attempt {i+1}:''')

        newWordList = dictionary[dictionary[0].str.contains(correct_pos_rex)==True]
        newWordList = newWordList[newWordList[0].str.contains(incorrect_pos_rex)==True]
        newWordList = newWordList[newWordList[0].str.contains(exist_in_rex)==True]
        newWordList = newWordList[newWordList[0].str.contains(not_exist_rex)==False]
        newWordList = newWordList[newWordList[0].isin(not_recognized_word_list)==False]
        
        newWordList["var"]=newWordList.apply(lambda x: 1-(sum([1 for y in x[0] if (x[0].count(y)-1) > 0]) / len(x[0])) , axis=1)
        newWordList["weight"]=newWordList.apply(lambda x: sum([1.5 for y in x[0] if y in ['e', 'a', 'i', 'o']]) + sum([1 for y in x[0] if y in ['t', 'n', 's', 'h', 'r']]), axis=1)
        
 
        newWordList = newWordList[newWordList["var"]==newWordList['var'].max()]
        newWordList = newWordList[newWordList["weight"]==newWordList['weight'].max()]
                
        str_try = newWordList.sample()[0].item()
        
        print(f'''\nInitial Regex Used:
        Correct Pos:          {correct_pos_rex}                    
        InCorrect Pos:        {incorrect_pos_rex}                    
        Contain:              {exist_in_rex}
        Does not Contain:     {not_exist_rex}
        Not Recognized words: {not_recognized_word_list}
        ''')
        
        print(f'''Try the word: {str_try}''')

        resp = ""
        while len(resp) != word_lengh or special_match(f'''^[r|g|y]{{{word_lengh}}}$''',resp)==False:
            resp = input(f'''Enter {word_lengh} char sequence (no spaces)
                            "g" - green 
                            "y" - yellow 
                            "r" - gray
                            'x' - not recognized word: ''')
            if (resp=="x"):
                not_recognized_word_list.append(str_try)
                newWordList = newWordList[newWordList[0].isin(not_recognized_word_list)==False]
                str_try = newWordList.sample()[0].item()                
                print(f'''\nNew word on Attempt {i+1}''')
                print(f'''Try the word: {str_try}''')
            
            
        if (resp == 'g' * word_lengh):
            print("\nCongrats, you won!")
            return
        
        for i in range(len(resp)):   
            if resp[i] == 'g':
                correct_pos_list[i] = str_try[i]
            if resp[i] == 'r':
                not_exist_list.append(str_try[i])
            if resp[i] == 'y':
                if (incorrect_pos_list[i] == "a-z"):
                    incorrect_pos_list[i] = "^" + str_try[i]
                else:
                    incorrect_pos_list[i] = incorrect_pos_list[i] + str_try[i]                     
                
                if (str_try[i] not in exist_in_list):
                    exist_in_list.append(str_try[i])
        
        
        correct_pos_rex   = "^"+ "".join( "[" + str(x) +"]" for x in correct_pos_list ) +"$"
        incorrect_pos_rex = "^"+ "".join( "[" + str(x) +"]" for x in incorrect_pos_list ) +"$"
        exist_in_rex      = "^.*" + "".join( "(?=.*[" + str(x) +"])" for x in exist_in_list )  +".*"
        not_exist_rex     = "^.*(?=.*[" + "".join(str(x) for x in not_exist_list )  +"]).*"
        

    
    print("\nSorry, you lose")
        


In [142]:
words = pd.read_csv('https://github.com/dwyl/english-words/raw/master/words_alpha.txt', header=None)

In [143]:
break_wordle(6,5,words)


###############################################################################################
Attempt 1:

Initial Regex Used:
        Correct Pos:          ^[a-z][a-z][a-z][a-z][a-z]$                    
        InCorrect Pos:        ^[a-z][a-z][a-z][a-z][a-z]$                    
        Contain:              ^.*.*
        Does not Contain:     ^.*(?=.*[.]).*
        Not Recognized words: []
        
Try the word: eoith


KeyboardInterrupt: Interrupted by user

In [132]:
l = ["a=z"] * 3
l

['a=z', 'a=z', 'a=z']